In [1]:
!pip install gurobipy  # install gurobipy, if not already installed

     |████████████████████████████████| 11.5 MB 5.4 MB/s 


# **Single Run**

In [12]:
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [3]:
class mid_mile:

    def __init__(self, K, M, R, N):

        #self.graph = graph
        self.K = K
        self.M = M
        self.R = R
        self.N = N                                       # no of nodes/warehouses

        self.V = np.zeros(shape=(self.K))           # the demand of each commodity K in route R
        self.v = np.zeros(shape=(self.N, self.N, self.M))

        self.A = np.zeros(shape=(self.N, self.N, self.M))
        self.F = np.zeros(shape=(self.N, self.N, self.M))
        
        self.origins = []
        self.destinations = []
        self.commodity_routes = []


        self.C = np.zeros(shape=(self.R))
        self.matrix = np.zeros(shape=(self.N, self.N, self.R))
        self.routes = []

        self.model = gp.Model()

    def add_variables(self):
        fy_list = []
        rk = []

        # decision variables: xr, ylm, vlm, flm
        for n1 in range(self.N):
            for n2 in range(self.N):
                for m in range(self.M):
                    for k in range(self.K):
                        for r in range(self.R):
                            fy_list.append((n1, n2, m, r, k))

        for r in range(self.R):
            for k in range(self.K):
                rk.append((r,k))

        fy_list = gp.tuplelist(fy_list)
        rk = gp.tuplelist(rk)

        self.x = self.model.addVars(rk, vtype=GRB.BINARY, name="x")
        self.y = self.model.addVars(fy_list, vtype=GRB.BINARY, name="y")
        self.f = self.model.addVars(fy_list, vtype=GRB.INTEGER, name="f")

        self.model.update()

    def add_init_constraints(self):
        self.add_constraints1()
        self.add_constraints2()
        self.add_constraints3()
        self.add_constraints4()
        self.add_constraints5()
        self.add_constraints6()


    def add_constraints1(self):
        # # each commodity can have only one route associated only in the feasible set of routes that can take it from 'ok' to 'dk'
        for k in range(self.K):
            lhs = 0
            for r in range(self.R):
                # if r in self.routes[k]:                     #if r is a feasible route, then it contribute
                #     lhs += self.x[r, k]
                # else:
                #     self.x[r, k] = 0                        #if r is not a feasible route, then it should be zero
                lhs += self.x[r, k]
            self.model.addConstr(lhs == 1, name="c1")

        self.model.update()


    def add_constraints2(self):
        ###### each route can have atmost one commodity #######
        lhs = 0
        for r in range(self.R):
            for k in range(self.K):
                lhs += self.x[r, k]
            self.model.addConstr(lhs <= 1,name="c2")
        
        self.model.update()


    def add_constraints3(self):
        # outflow >= inflow
        for k in range(self.K):
            #for r in self.routes[k]:
            for r in range(self.R):
                route = self.routes[r]

                if len(route) > 1:
                    for l in range(len(route)-1):
                        left_node = route[l][0]
                        central_node = route[l][1]
                        right_node = route[l+1][1]
    
                        inflow = 0
                        for m in range(self.M):
                            inflow += self.f[left_node, central_node, m, r, k] * self.v[left_node][central_node][m]
                        
                        outflow = 0
                        for m in range(self.M):
                            outflow += self.f[central_node, right_node, m, r, k] * self.v[central_node][right_node][m]
                        
                        self.model.addConstr(outflow - inflow >= 0, name="control_volume_constraint")
        
        self.model.update()


    def add_constraints4(self):
        # assign the Demand to each route selected
        for k in range(self.K):
            for r in range(self.R):
                assignment = self.V[k] * self.x[r, k]
                start_leg = self.routes[r][0]

                init_flow = 0
                for m in range(self.M):
                    init_flow += self.f[start_leg[0], start_leg[1], m, r, k] * self.v[start_leg[0]][start_leg[1]][m]
                                                 
                self.model.addConstr(init_flow - assignment >= 0, name="demand_assignment_constraint")
        
        self.model.update()


    def add_constraints5(self):
        for n1 in range(self.N):
            for n2 in range(self.N):
                for m in range(self.M):
                    lhs = 0
                    activate = 0
                    for r in range(self.R):
                        for k in range(self.K):
                            lhs += self.f[n1, n2, m, r, k]

                    self.model.addConstr(lhs - self.F[n1][n2][m] <= 0, name="c5")
            
        self.model.update()


    def add_constraints6(self):
        # constraint y[l, m, r, k]

        for r in range(self.R):
            for k in range(self.K):
                for n1 in range(self.N):
                    for n2 in range(self.N):
                        for m in range(self.M):    
                            rhs = self.x[r, k] * self.matrix[n1][n2][r]
                            lhs = self.y[n1, n2, m, r, k]
                            self.model.addConstr(lhs == rhs, name="c6")
        
        self.model.update()


    def objective(self):
        term2 = 0
        for r in range(self.R):
            for n1 in range(self.N):
                for n2 in range(self.N):
                    for m in range(self.M):
                        for k in range(self.K):
                            term2 += self.A[n1][n2][m]*self.f[n1, n2, m, r, k]
            
        term1 = 0
        for k in range(self.K):
            for r in range(self.R):
                term1 += self.x[r, k] * self.C[r]
        
        obj = term1 + term2

        self.model.setObjective(obj, GRB.MINIMIZE)

        self.model.update()

    def process(self):
        self.status = self.model.optimize()

    def get_solution(self):
        self.y_sol = test.model.getAttr('X', test.y)
        self.f_sol = test.model.getAttr('X', test.f)
        self.x_sol = test.model.getAttr('X', test.x)
    

#########################################################################################################################################################
#################################### >>>>>>>>>>>>>>>>>>     SIMULATION  FUNCTIONS    <<<<<<<<<<<<<<<<<< #################################################
#########################################################################################################################################################

    def check_constraints(self, V_new):

        self.V_new = V_new

        ck1 = self.check_constraint1()
        ck2 = self.check_constraint2()
        ck3 = self.check_constraint3()
        ck4 = self.check_constraint4()
        ck5 = self.check_constraint5()

        ck = [ck1, ck2, ck3, ck4, ck5]

        return ck
    
    def check_constraint1(self):
        lhs = 0
        for k in range(self.K):
            for r in range(self.R):
                # if r in self.routes[k]:                     #if r is a feasible route, then it contribute
                #     lhs += self.x[r, k]
                # else:
                #     self.x[r, k] = 0                        #if r is not a feasible route, then it should be zero
                lhs += self.x_sol[r, k]

            if lhs != 1:
                return False
        
        return True

    def check_constraint2(self):
        ###### each route can have atmost one commodity #######
        lhs = 0
        for r in range(self.R):
            for k in range(self.K):
                lhs += self.x_sol[r, k]
            
            if lhs > 1:
                return False
        
        return True

    def check_constraint3(self):
        # outflow >= inflow
        for k in range(self.K):
            #for r in self.routes[k]:
            for r in range(self.R):
                route = self.routes[r]

                if len(route) > 1:
                    for l in range(len(route)-1):
                        left_node = route[l][0]
                        central_node = route[l][1]
                        right_node = route[l+1][1]
    
                        inflow = 0
                        for m in range(self.M):
                            inflow += self.f_sol[left_node, central_node, m, r, k] * self.v[left_node][central_node][m]
                        
                        outflow = 0
                        for m in range(self.M):
                            outflow += self.f_sol[central_node, right_node, m, r, k] * self.v[central_node][right_node][m]
                    
                        if (outflow - inflow) < 0:
                            return False
        
        return True

    def check_constraint4(self):
        # assign the Demand to each route selected
        for k in range(self.K):
            for r in range(self.R):
                assignment = self.V_new[k] * self.x_sol[r, k]
                start_leg = self.routes[r][0]

                init_flow = 0
                for m in range(self.M):
                    init_flow += self.f_sol[start_leg[0], start_leg[1], m, r, k] * self.v[start_leg[0]][start_leg[1]][m]
                
                if (init_flow - assignment) < 0:
                    return False
        
        return True

    def check_constraint5(self):
        for r in range(self.R):
            for n1 in range(self.N):
                for n2 in range(self.N):
                    for m in range(self.M):
                        lhs = 0
                        for k in range(self.K):
                            lhs += self.f_sol[n1, n2, m, r, k]
                            rhs = self.F[n1][n2][m] * self.y_sol[n1, n2, m, r, k]

                        if (lhs - rhs) > 0:
                            return False
        
        return True


#########################################################################################################################################################
# this functions is wrong; because it shows the route only if the ok and dk are directly connected -- we may have to implement standard path planning algorithms
#########################################################################################################################################################
    def get_commodity_routes(self):
        for k in range(self.K):
            rs = []
            ok = self.origins[k]
            dk = self.destinations[k]
            
            for r in range(self.R):
                if self.routes[r][0][0] == ok and self.routes[r][-1][1] == dk:
                    self.commodity_routes.append(r)
    
    def obtain_routes(self):
        for r in range(self.R):
            r_path = []
            for n1 in range(self.N):
                for n2 in range(self.N):
                    if self.matrix[n1][n2][r] == 1:
                        r_path.append((n1, n2))
            
            self.routes.append(r_path)
#########################################################################################################################################################
#########################################################################################################################################################

In [ ]:
# Input model parameter values

K = 1
M = 1                       # only one mode of transport
N = 3
R = 2


origins_k = [0]
destinations_k = [2]

V = np.zeros(shape=(K))
V[0] = 700


######## to be implemented ========>>>>>>>>>>>>> it can also depend on mode of transport ###########
C = [10, 5]

v = np.zeros(shape=(N, N, M))
v[0][1][0] = 50
v[1][2][0] = 45
v[0][2][0] = 30


F = np.zeros(shape=(N, N, M))
F[0][1][0] = 15
F[1][2][0] = 20
F[0][2][0] = 15


A = np.ones(shape=(N, N, M))
A[0][1][0] = 9
A[1][2][0] = 2
A[0][2][0] = 6


graph = np.zeros(shape=(N, N, R))
graph[0][1][0] = 1
graph[1][2][0] = 1
graph[0][2][1] = 1

In [ ]:
test = mid_mile(K, M, R, N)

test.V = V
test.v = v
test.F = F
test.A = A
test.C = C
test.matrix = graph
test.origins = origins_k
test.destinations = destinations_k

test.obtain_routes()
test.get_commodity_routes()

In [ ]:
print(test.routes)
print(test.commodity_routes)

[[(0, 1), (1, 2)], [(0, 2)]]
[0, 1]


In [6]:
########### for slides demo ##########

'''
Let k=1
Origin- Node(0)
destination - Node(3)
Demand - 20 shipments / day
Possible routes -                                                    {(0,3), (0,2,3), (0,1,2,3)}

Modes - {Road, Seaway, Airways}
Cost associated with Route - {50, 10, 15} units

A[0][3][air] = 10000  per load 
A[0][3][road] = 500
A[0][2][air] = 10000
A[2][3][road] = 100
A[0][1][air] =  7500
A[1][2][road] = 300
A[0][2][road] = 600
A[2][3][air] = 7500

F[0][3][air] = 5
F[0][3][road] = 20
F[0][2][road] = 15
F[2][3][air] = 2
F[2][3][road] = 18
F[0][1][air] = 3
F[0][1][road] = 15
F[1][2][road] = 20

v[:][:][air] = 4000
v[:][:][road] = 1500


'''

K = 1
M = 2                       # only one mode of transport
N = 4
R = 3


origins_k = [0]
destinations_k = [3]

V = np.zeros(shape=(K))
V[0] = 2680


######## to be implemented ========>>>>>>>>>>>>> it can also depend on mode of transport ###########
C = [50, 10, 15]

v = np.zeros(shape=(N, N, M))
v[0][3][0] = 400
v[0][3][1] = 150
v[0][2][1] = 150
v[2][3][0] = 400
v[2][3][1] = 150
v[0][1][0] = 400
v[0][1][1] = 150
v[1][2][1] = 150


F = np.zeros(shape=(N, N, M))
F[0][3][0] = 5
F[0][3][1] = 15
F[0][2][1] = 15
F[2][3][0] = 2
F[2][3][1] = 18
F[0][1][0] = 3
F[0][1][1] = 15
F[1][2][1] = 20


A = np.ones(shape=(N, N, M))
A[0][3][0] = 100000
A[0][3][1] = 5000
A[0][2][0] = 10000
A[2][3][1] = 100
A[0][1][0] = 7500
A[0][1][1] = 300
A[1][2][0] = 7500
A[1][2][1] = 100
A[0][2][1] = 600
A[2][3][0] = 7500


graph = np.zeros(shape=(N, N, R))
graph[0][3][0] = 1
graph[0][2][1] = 1
graph[2][3][1] = 1
graph[0][1][2] = 1
graph[1][2][2] = 1
graph[2][3][2] = 1

In [ ]:
K = 2
M = 2                       # only one mode of transport
N = 4
R = 3


origins_k = [0]
destinations_k = [3]

V = np.zeros(shape=(K))
V[0] = 2680


######## to be implemented ========>>>>>>>>>>>>> it can also depend on mode of transport ###########
C = [50, 10, 15]

v = np.zeros(shape=(N, N, M))
v[0][3][0] = 400
v[0][3][1] = 150
v[0][2][1] = 150
v[2][3][0] = 400
v[2][3][1] = 150
v[0][1][0] = 400
v[0][1][1] = 150
v[1][2][1] = 150


F = np.zeros(shape=(N, N, M))
F[0][3][0] = 5
F[0][3][1] = 15
F[0][2][1] = 15
F[2][3][0] = 2
F[2][3][1] = 18
F[0][1][0] = 3
F[0][1][1] = 15
F[1][2][1] = 20


A = np.ones(shape=(N, N, M))
A[0][3][0] = 100000
A[0][3][1] = 5000
A[0][2][0] = 10000
A[2][3][1] = 100
A[0][1][0] = 7500
A[1][2][1] = 300
A[0][2][1] = 600
A[2][3][0] = 7500


graph = np.zeros(shape=(N, N, R))
graph[0][3][0] = 1
graph[0][2][1] = 1
graph[2][3][1] = 1
graph[0][1][2] = 1
graph[1][2][2] = 1
graph[2][3][2] = 1

In [7]:
test = mid_mile(K, M, R, N)

test.V = V
test.v = v
test.F = F
test.A = A
test.C = C
test.matrix = graph
test.origins = origins_k
test.destinations = destinations_k

test.obtain_routes()

test.commodity_routes = [[0,1], [0, 1]]



test.add_variables()

test.add_constraints1()
test.add_constraints2()
test.add_constraints3()
test.add_constraints4()
test.add_constraints5()
test.add_constraints6()


test.objective()
test.process()

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 138 rows, 195 columns and 230 nonzeros
Model fingerprint: 0xf583d1fe
Variable types: 0 continuous, 195 integer (99 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 134 rows and 187 columns
Presolve time: 0.00s
Presolved: 4 rows, 8 columns, 12 nonzeros
Variable types: 0 continuous, 8 integer (1 binary)
Found heuristic solution: objective 30015.000000

Root relaxation: objective 2.241500e+04, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 22415.0000    0    1 30015.0000 224

In [8]:
print(test.routes)

[[(0, 3)], [(0, 2), (2, 3)], [(0, 1), (1, 2), (2, 3)]]


In [9]:
y_sol = test.model.getAttr('X', test.y)
f_sol = test.model.getAttr('X', test.f)
x_sol = test.model.getAttr('X', test.x)

In [10]:
#x_sol = test.model.getAttr('X', test.x)
print(x_sol[0, 0], '   ', x_sol[1, 0], '   ', x_sol[2, 0])

0.0     0.0     1.0


In [11]:
y = 0

for r in range(R):
    for k in range(K):
        for n1 in range(N):
            for n2 in range(N):
                for m in range(M):
                    if f_sol[n1, n2, m, r, k] != 0:
                        print(str((n1, n2, m , r, k)) + " : {}".format(f_sol[n1, n2, m, r, k]))

(0, 1, 0, 2, 0) : 3.0
(0, 1, 1, 2, 0) : 10.0
(1, 2, 1, 2, 0) : 18.0
(2, 3, 1, 2, 0) : 18.0


In [ ]:
print(f_sol[0, 1, 0, 0, 0])
print(f_sol[1, 2, 0, 0, 0])

-0.0
-0.0


In [ ]:
for r in range(R):
    for n1 in range(N):
        for n2 in range(N):
            for m in range(M):
                for k in range(K):
                    if y_sol[n1, n2, m, r, k] == 1:
                        print((n1, n2, m , r, k))

(0, 3, 0, 0, 0)
(0, 3, 1, 0, 0)


# **Simulation**

In [ ]:
import matplotlib.pyplot as plt
from math import floor

In [ ]:
# simulation for 100 times

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

# Input model parameter values
K = 1
M = 1                       # only one mode of transport
N = 3
R = 2


origins_k = [0]
destinations_k = [2]

V = np.zeros(shape=(K))
V[0] = 700


######## to be implemented ========>>>>>>>>>>>>> it can also depend on mode of transport ###########
C = [10, 5]

v = np.zeros(shape=(N, N, M))
v[0][1][0] = 50
v[1][2][0] = 45
v[0][2][0] = 60


F = np.zeros(shape=(N, N, M))
F[0][1][0] = 15
F[1][2][0] = 20
F[0][2][0] = 15


A = np.ones(shape=(N, N, M))
A[0][1][0] = 9
A[1][2][0] = 2
A[0][2][0] = 6


graph = np.zeros(shape=(N, N, R))
graph[0][1][0] = 1
graph[1][2][0] = 1
graph[0][2][1] = 1

V_mu = V
V_sigma = V/10

# simulation first run
sim_model = mid_mile(K, M, R, N)

sim_model.V = V
sim_model.v = v
sim_model.F = F
sim_model.A = A
sim_model.C = C
sim_model.matrix = graph
sim_model.origins = origins_k
sim_model.destinations = destinations_k

sim_model.obtain_routes()
sim_model.commodity_routes = [[0,1]]

sim_model.add_variables()
sim_model.add_init_constraints()
sim_model.objective()

sim_model.process()

sim_model.get_solution()

sample_size = 1000

# uncertainity analysis
for i in range(sample_size):
    V_new = np.zeros(shape=(K))

    for k in range(K):
        v_k = np.random.normal(V_mu[k], V_sigma[k])
        v_k = floor(v_k)

        V_new[k] = v_k

    ck1, ck2, ck3, ck4, ck5 = sim_model.check_constraints(V_new)

    c1.append(ck1)
    c2.append(ck2)
    c3.append(ck3)
    c4.append(ck4)
    c5.append(ck5)

suc = 0
for ele in c4:
    suc += ele

print("\n Success rate: \n",suc/sample_size*100)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 24 rows, 38 columns and 35 nonzeros
Model fingerprint: 0x1b0bfddc
Variable types: 0 continuous, 38 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 24 rows and 38 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 77 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.700000000000e+01, best bound 7.700000000000e+01, gap 0.0000%

 Success rate: 
 60.3


In [ ]:
print("\n Success rate: \n",suc)

In [ ]:
y_sol = sim_model.model.getAttr('X', test.y)
f_sol = sim_model.model.getAttr('X', test.f)
x_sol = sim_model.model.getAttr('X', test.x)

In [ ]:
#x_sol = test.model.getAttr('X', test.x)
print(x_sol[0, 0], '   ', x_sol[1, 0])

for r in range(R):
    for k in range(K):
        for n1 in range(N):
            for n2 in range(N):
                for m in range(M):
                    if f_sol[n1, n2, m, r, k] != 0:
                        print(str((n1, n2, m , r, k)) + " : {}".format(f_sol[n1, n2, m, r, k]))

0.0     1.0
(0, 2, 0, 1, 0) : 12.0


# **Upgradation**

In [ ]:
def assign_fValues(f_sol, sample_value):
    global N, M, R, K
    global f_values
    for n1 in range(N):
        for n2 in range(N):
            for m in range(M):
                for r in range(R):
                    for k in range(K):
                        f_values[sample_value][n1][n2][m][r][k] = f_sol[n1, n2, m, r, k]

In [ ]:
def get_fAverage(f_mat):
    global N, M, R, K, sample_size
    f_avg = np.zeros(shape=(N, N, M, R, K))
    
    for n1 in range(N):
        for n2 in range(N):
            for m in range(M):
                for r in range(R):
                    for k in range(K):
                        val = 0
                        for s in range(sample_size):
                            val += f_mat[s][n1][n2][m][r][k]
                        f_avg[n1][n2][m][r][k] = val/sample_size
    
    return f_avg

In [ ]:
def get_xMode(x_mat):
    global R, sample_size
    x = np.zeros(shape=(K, R))
    for s in range(sample_size):
    
    pass

IndentationError: ignored

In [ ]:
# Upgradation

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

# Input model parameter values
K = 1
M = 1                       # only one mode of transport
N = 3
R = 2


origins_k = [0]
destinations_k = [2]

V = np.zeros(shape=(K))
V[0] = 700


######## to be implemented ========>>>>>>>>>>>>> it can also depend on mode of transport ###########
C = [10, 5]

v = np.zeros(shape=(N, N, M))
v[0][1][0] = 50
v[1][2][0] = 45
v[0][2][0] = 60


F = np.zeros(shape=(N, N, M))
F[0][1][0] = 15
F[1][2][0] = 20
F[0][2][0] = 15


A = np.ones(shape=(N, N, M))
A[0][1][0] = 9
A[1][2][0] = 2
A[0][2][0] = 6


graph = np.zeros(shape=(N, N, R))
graph[0][1][0] = 1
graph[1][2][0] = 1
graph[0][2][1] = 1

V_mu = V
V_sigma = V/2

sample_size = 10


# final results variables
x_values = np.zeros(shape=(sample_size, R, K))
f_values = np.zeros(shape=(sample_size, N, N, M, R, K))

# uncertainity analysis
for i in range(sample_size):
    V_new = np.zeros(shape=(K))

    for k in range(K):
        v_k = np.random.normal(V_mu[k], V_sigma[k])
        v_k = floor(v_k)

        V_new[k] = v_k
    
    print(V_new)
    
    # run the optimiser on new demand
    upg_model = mid_mile(K, M, R, N)

    upg_model.V = V_new
    upg_model.v = v
    upg_model.F = F
    upg_model.A = A
    upg_model.C = C
    upg_model.matrix = graph
    upg_model.origins = origins_k
    upg_model.destinations = destinations_k

    upg_model.model.Params.LogToConsole = 0

    upg_model.obtain_routes()
    upg_model.commodity_routes = [[0,1]]

    upg_model.add_variables()
    upg_model.add_init_constraints()
    upg_model.objective()

    upg_model.process()
    upg_model.get_solution()

    assign_fValues(f_sol, i)
    
    # for k in range(K):
    #     for r in range(R):
    #         if x_sol[r, k] == 1:
    #             k_route = r
    #             break
        
    #     x_values[i][k] = k_route

    # store the results

    del upg_model


f_avg = get_fAverage(f_values)

suc = 0
for ele in c4:
    suc += ele

print("\n Success rate: \n",suc/sample_size*100)

for r in range(R):
    for k in range(K):
        for n1 in range(N):
            for n2 in range(N):
                for m in range(M):
                    if f_avg[n1][n2][m][r][k] != 0:
                        print(str((n1, n2, m , r, k)) + " : {}".format(f_avg[n1][n2][m][r][k]))

[-89.]
[1044.]
[614.]
[1151.]
[908.]
[1307.]
[485.]
[525.]
[1306.]
[844.]

 Success rate: 
 0.0
(0, 2, 0, 1, 0) : 13.0


# **Diagnosis**

In [ ]:
# do IIS
print('The model is infeasible; computing IIS')
test.model.computeIIS()
if test.model.IISMinimal:
    print('IIS is minimal\n')
else:
    print('IIS is not minimal\n')
print('\nThe following constraint(s) cannot be satisfied:')
for c in test.model.getConstrs():
    if c.IISConstr:
        print('%s' % c.ConstrName)